# Sonification of seismic data

Small track loop (drone + beats) made exclusively from seismic data.

See also https://jbrussell.github.io/eilive2020/part2a_sonification/

In [ ]:
import ipytone
import ipygany
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import pyvista.examples as pv_examples
from obspy import read
from obspy.signal.trigger import classic_sta_lta

In [ ]:
t = ipytone.transport

## Drone sound

In [ ]:
st = read("https://examples.obspy.org/BW.KW1..EHZ.D.2011.037")
data = st.select(id="BW.KW1..EHZ")[0].data

data_norm = data / (data.max() - data.min())
drone_sample = data_norm[0:300_000] + 0.25 * 2

In [ ]:
plt.plot(drone_sample);

In [ ]:
drone_buffer = ipytone.AudioBuffer(drone_sample)

In [ ]:
drone_player = ipytone.Player(drone_buffer, volume=-2)
drone_filter = ipytone.Filter(frequency=8000)
drone_comp = ipytone.Compressor()
drone_delay = ipytone.PingPongDelay()

drone_player.chain(
    drone_comp, drone_filter, drone_delay, ipytone.destination
)

In [ ]:
drone_player.fade_in = 0.02
drone_player.fade_out = 0.5
drone_player.loop = True
drone_player.playback_rate = 0.08

drone_comp.ratio.value = 20
drone_filter.frequency.value = 2000
drone_filter.rolloff = -24
drone_delay.delay_time.value = 2
drone_delay.feedback.value = 0.2

## Beat

In [ ]:
trace = read("https://examples.obspy.org/ev0_6.a01.gse2")[0]

# auto find trigger
df = trace.stats.sampling_rate
cft = classic_sta_lta(trace.data, int(5 * df), int(10 * df))

raw_sample = trace.data[np.argmax(cft > 1.75):]
drum_sample = raw_sample / (raw_sample.max() - raw_sample.min()) * 1.5

In [ ]:
plt.plot(drum_sample);

In [ ]:
drum_buffer = ipytone.AudioBuffer(drum_sample)

In [ ]:
comp = ipytone.Compressor()
delay = ipytone.PingPongDelay()
reverb = ipytone.Reverb()
drums = ipytone.Sampler({"A5": drum_buffer}, volume=-3)
drums.chain(comp, delay, reverb, ipytone.destination)

In [ ]:
drums.attack = 0.02
drums.release = 0.8

comp.attack.value = 0.3
comp.ratio.value = 5

delay.wet.value = 0.1

reverb.decay = 4
reverb.wet.value = 0.3

In [ ]:
def clb(time, note):
    drums.trigger_attack_release(note.note, 0.1, time=time, velocity=note.velocity)

In [ ]:
# kick / snare
beat_notes = [
    {"time": "0:0:0", "note": "E3", "velocity": 1},
    {"time": "0:2:0", "note": "B5", "velocity": 1},
    {"time": "0:3:2", "note": "E3", "velocity": 0.3},
    {"time": "1:0:0", "note": "E3", "velocity": 1},
    {"time": "1:0:2", "note": "B5", "velocity": 1},
    {"time": "1:2:0", "note": "B5", "velocity": 1},
    {"time": "1:3:2", "note": "E3", "velocity": 0.2},
]

# hi-hats
beat_notes += [
    {"time": f"{i}:{j}:{k}", "note": 1e4, "velocity": 1 - k / 4}
    for k in [0, 2]
    for j in range(4)
    for i in range(2)
]

In [ ]:
beat_part = ipytone.Part(callback=clb, events=beat_notes)

In [ ]:
beat_part.loop = True
beat_part.loop_start = 0
beat_part.loop_end = "2m"

## Bass synth

In [ ]:
synth1 = ipytone.MonoSynth(volume=-4).to_destination()
synth2 = ipytone.MonoSynth(volume=-7).to_destination()

In [ ]:
synth1.oscillator.type = "sine"
synth1.envelope.attack = 0.02

synth2.oscillator.type =  "fatsawtooth"
synth2.filter_envelope.attack = 6
synth2.filter.q.value = 4
synth2.filter_envelope.sustain = 0.2

In [ ]:
def synth_clb(time, note):
    synth1.trigger_attack_release(note.note, note.duration, time=time)
    synth2.trigger_attack_release(note.note, note.duration, time=time)

In [ ]:
synth_part = ipytone.Part(
    callback=synth_clb,
    events=[
        {"time": "0:0:0", "note": "G1", "duration": "1m"},
        {"time": "1:0:0", "note": "B#1", "duration": "8n"},
        {"time": "1:0:2", "note": "A#1", "duration": "2n"},
        {"time": "1:3:2", "note": "B#1", "duration": "8n"},
        {"time": "2:0:0", "note": "D#1", "duration": "1m"},
        {"time": "3:0:0", "note": "G1", "duration": "2n"},
        {"time": "3:2:0", "note": "E#1", "duration": "2n"},
    ]
)

In [ ]:
synth_part.loop = True
synth_part.loop_start = 0
synth_part.loop_end = "4m"

## Pluck Synth

In [ ]:
vibrato = ipytone.Vibrato()
synth3 = ipytone.PluckSynth(volume=-19)
synth3.chain(vibrato, reverb)
lfo = ipytone.LFO(min=10, max=20).start()
lfo.connect(vibrato.frequency)

In [ ]:
vibrato.wet.value = 0.8
synth3.resonance = 0.75
synth3.dampening = 3000
synth3.attack_noise = 6

In [ ]:
def synth3_clb(time, value):
    synth3.trigger_attack_release(value, 0.1, time=time)

In [ ]:
seq = ipytone.Sequence(
    callback=synth3_clb,
    events=[["G#3", "G#1"], "G#3", "G#3", "G#3", "G#3", "G#3", "G#3", ["G#3", "G#4"]],
    subdivision="4n",
)

## Animated 3D topography (just for fun)

Topography exagerration factor will follow the beats.

In [ ]:
pvmesh = pv_examples.download_topo_land()
ugrid = pvmesh.cast_to_unstructured_grid()
mesh = ipygany.PolyMesh.from_vtk(ugrid)

pvsphere = pv.Sphere(radius=0.999)
usphere = pvsphere.cast_to_unstructured_grid()
sphere = ipygany.PolyMesh.from_vtk(usphere)

colored_mesh = ipygany.IsoColor(
    mesh, min=0.0, max=6527.0,
    colormap=ipygany.colormaps.Oranges
)
warped_mesh = ipygany.WarpByScalar(
    colored_mesh, input='altitude', factor=5e-5
)

scene = ipygany.Scene([sphere, warped_mesh])

In [ ]:
meter = ipytone.DCMeter()
follower_node = ipytone.Follower()
add_node = ipytone.Add(addend=1e-4)
mult_node = ipytone.Multiply(factor=6e-4)
drums.chain(follower_node, add_node, mult_node, meter)

In [ ]:
follower_node.smoothing = 1.4

In [ ]:
link = meter.schedule_jsdlink((warped_mesh, "factor"), transport=True)

In [ ]:
meter2 = ipytone.DCMeter()
mult2_node = ipytone.Multiply(factor=10)
synth1.filter_envelope.chain(mult2_node, meter2)

In [ ]:
link2 = meter2.schedule_jsdlink((colored_mesh, "max"), transport=True)

## Play!

In [ ]:
ipytone.destination.volume.value = -3

In [ ]:
drone_player.start()

In [ ]:
scene

In [ ]:
t.start()
beat_part.start(0)
synth_part.start("4m")
seq.start("12m")

In [ ]:
t.stop()
beat_part.stop()
synth_part.stop()
seq.stop()

In [ ]:
drone_player.stop()

## Clean-up

In [ ]:
link.unlink()
link2.unlink()

In [ ]:
t.cancel()

In [ ]:
beat_part.dispose()
synth_part.dispose()
seq.dispose()

In [ ]:
drone_player.dispose()
drone_comp.dispose()
drone_filter.dispose()
drone_delay.dispose()
drums.dispose()
comp.dispose()
delay.dispose()
reverb.dispose()
meter.dispose()
follower_node.dispose()
mult_node.dispose()
vibrato.dispose()
lfo.dispose()
synth1.dispose()
synth2.dispose()
synth3.dispose()